# Mean Risk Optimisation
## Loading Packages

In [1]:
using PortfolioOptimiser, TimeSeries, DataFrames, CSV, Clarabel

## Loading sample data

In [9]:
# Reading the data.
A = TimeArray(CSV.File("../test/assets/stock_prices.csv"); timestamp=:date)
# Computing returns.
Y = percentchange(A)
# Turning into a dataframe and rop missing values.
returns = dropmissing!(DataFrame(Y))
display(returns[1:5, :])

Row,timestamp,GOOG,AAPL,FB,BABA,AMZN,GE,AMD,WMT,BAC,GM,T,UAA,SHLD,XOM,RRC,BBY,MA,PFE,JPM,SBUX
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2014-09-22,-0.0146122,0.000990734,-0.0142472,-0.042603,-0.0205844,-0.00798774,-0.0131234,-0.00689774,0.0047195,-0.014732,0.000845735,-0.0283585,-0.0101156,-0.0059719,-0.0181691,-0.02716,-0.0144927,-0.00723714,-0.00327289,-0.0193242
2,2014-09-23,-0.0106236,0.0156342,0.019401,-0.0302592,-0.00268103,-0.00230073,-0.018617,-0.00930391,0.00117438,-0.00657874,-0.00676033,0.00616814,0.0102189,-0.00528286,-0.00384323,-0.00920662,-0.00853472,-0.00430752,0.000492726,-0.00857916
3,2014-09-24,0.0118046,-0.00867105,0.00319326,0.0390043,0.0141519,-0.00345899,0.00271003,0.0195765,0.00762483,0.012944,0.00397038,0.0269139,-0.0458815,-0.00218697,-0.0148616,0.0188848,0.0203943,0.00865238,0.0113225,0.0183884
4,2014-09-25,-0.0219902,-0.0381327,-0.0168067,-0.018218,-0.0191341,-0.0146548,-0.0189189,-0.0124545,-0.0192082,-0.0231799,-0.0090394,-0.0164531,-0.0283983,-0.0163845,-0.0236437,-0.021477,-0.0302399,-0.0108875,-0.0240143,-0.0159322
5,2014-09-26,0.0035475,0.0294266,0.0203315,0.017319,0.00397601,0.00313095,-0.00826446,0.00486075,0.0106821,0.00912671,0.00570092,0.0336047,-0.0315667,0.0125198,0.00579415,-0.00962121,0.00455039,-0.00867243,0.00681621,0.0141663


## Minimal Working Example

In [14]:
# We instantiate the portfolio instance.
portfolio = Portfolio(;
    # Returns dataframe
    returns = returns,
    # Solvers is a Dictionary.
    solvers = Dict(
        # We will use the Clarabel.jl optimiser. In this case we use a dictionary
        # for the value, but we can also use named tuples, all we need are key-value
        # pairs.
        :Clarabel => Dict(
            # :solver key must contain the optimiser structure.
            :solver => Clarabel.Optimizer,
            # :params key is optional, but if it is present, it defines solver-specific
            # attributes/configurations. This often needs to be a dictionary as the 
            # solver attributes are usually strings.
            :params => Dict("verbose" => false, "max_step_fraction" => 0.75),
        ),
    ),
)

Before we optimise we need to compute some statistics. This function has a *ton* of keyword arguments for everything you'd like to do. See the docs for more.

In [15]:
asset_statistics!(portfolio)

We can then optimise our portfolio.

In [13]:
w = opt_port!(portfolio)

Row,tickers,weights
,String,Float64
1,GOOG,0.0325331
2,AAPL,0.0753982
3,FB,0.0525912
4,BABA,0.0338242
5,AMZN,0.0259076
6,GE,0.0661884
7,AMD,0.0159081
8,WMT,0.0799032
9,BAC,0.0305654
